# About 

This notebook is a usecase about using **LLM Agents** to read a **pdf** and make a summary. 

Here, I used Microsoft’s **AutoGen**, a framework for performing tasks using LLM agents.

# 1. Settings

### Packages

In [ ]:
# install pyautogen package if you haven't installed
# %pip install pyautogen

In [10]:
import json
import os

import chromadb

# autogen 
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# tell autogen do not use docker for autogen
os.environ["AUTOGEN_USE_DOCKER"] = "False"
os.environ["TOKENIZERS_PARALLELISM"] = "False"

### Variables

In [11]:
#----------------#
# variables that require changes
#----------------#
llm_model_id ="llama2"

# file path
# the pdf file here is from "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10872502/"
docs_path =  ["../data/pdf/West_2024.pdf"]

# specify file format 
custom_text_types = ["pdf"]

# accepted file formats are 
# ['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']

### LLM Model API Endpoint

In [12]:
# model endpoint 
# in this case, it is the Ollama local host address. No api_key 
config_list = [
    {
        'base_url': "http://localhost:11434/v1",  
        'model': llm_model_id,
        'api_key': "NULL",
    }
]

# llm config
llm_config = {
    "config_list": config_list, 
    "cache_seed": 42,
    "timeout": 1200
}



# 2. Create Two Retrieval-augmented Agents

### a) User Proxy Agent: who translates the user's request into messages and context
It retrieves document chunks based on the embedding similarity, and sends them along with the question to the Retrieval-Augmented Assistant.

In [13]:
# 1. create the RetrieveUserProxyAgent instance named "ragproxyagent"
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": docs_path,
        "custom_text_types": custom_text_types,
        "chunk_token_size": 2000,
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "must_break_at_empty_line": False,
        "embedding_model": "all-mpnet-base-v2",
        "get_or_create": True,  # set to False if you don't want to reuse an existing collection, but you'll need to remove the collection manually
    },
    code_execution_config=False,  # set to False if you don't want to execute the code
)

# By default, the human_input_mode is "ALWAYS", which means the agent will ask for human input at every step. We set it to "NEVER" here.
# `docs_path` is the path to the docs directory. It can also be the path to a single file, or the url to a single file. 
# By default, it is set to None, which works only if the collection is already created.
# `task` indicates the kind of task we're working on. In this example, it's a `qa` task.
# `chunk_token_size` is the chunk token size for the retrieve chat. By default, it is set to `max_tokens * 0.6`, here we set it to 2000.
# `custom_text_types` is a list of file types to be processed. Default is `autogen.retrieve_utils.TEXT_FORMATS`.

### b) Retrieve Assistant Agent: who generates the output
 * It employs an LLM to generate text (or code) as answers based on the question and context provided. 
 * If the LLM is unable to produce a satisfactory response, it is instructed to reply with “Update Context” to the Retrieval-Augmented User Proxy.

In [14]:
# 2. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config
)


# 3. Chat

### User's input

In [15]:
# given a question, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
question = "What is eculizumab?"


### Agents in action

In [16]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# answer
answer = ragproxyagent.initiate_chat(
    assistant, 
    problem=question,
    silent=True
)  

Trying to create collection.
doc_ids:  [['doc_3', 'doc_10', 'doc_9', 'doc_11', 'doc_0', 'doc_8', 'doc_7', 'doc_12', 'doc_5', 'doc_2', 'doc_6', 'doc_4', 'doc_1', 'doc_232', 'doc_124', 'doc_75', 'doc_119', 'doc_78', 'doc_200', 'doc_240']]
Adding doc_id doc_3 to context.
Adding doc_id doc_10 to context.
Adding doc_id doc_9 to context.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Result


In [17]:
answer.summary.splitlines()

['Complement is a crucial part of the immune system that helps to protect against infection and disease. There are three main branches of the complement system: the classical pathway, the alternative pathway, and the lectin pathway. Each branch has its own specific triggers and outcomes. The complement system plays a key role in inflammation, immune response, and tissue repair.',
 '',
 'Complement-mediated diseases are disorders that result from problems with the complement system. Examples of these diseases include paroxysmal nocturnal hemoglobinuria (PNH), atypical hemolytic uremic syndrome (aHUS), and geographic atrophy secondary to age-related macular degeneration (GA).',
 '',
 'Approved complement drugs currently exist for the treatment of some complement-mediated diseases. These include eculizumab, ravulizumab, and pegcetacoplan. Eculizumab is an anti-C5 drug that treats PNH, a rare genetic disorder that causes hemolytic anemia. Ravulizumab is also an anti-C5 drug that treats aHU